In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import json

In [3]:
import sys
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

In [4]:
with open('../train.json', 'r') as f:
    train = json.load(f)
with open('../test.json', 'r') as f:
    test = json.load(f)

In [5]:
vec = []
for i in train:
    [vec.append(j) for j in i['ingredients']]

In [6]:
cats = np.unique((np.array(vec)))

In [7]:
from scipy.sparse import csr_matrix

In [8]:
X = csr_matrix(map(lambda y: np.array(map(lambda x: 1 if x in y['ingredients'] else 0, cats), dtype='int8'), train))

In [9]:
X.shape

(39774, 6714)

In [10]:
target = map(lambda x: x['cuisine'], train)

In [11]:
b, y = np.unique(target, return_inverse=True)

In [12]:
test_sp = csr_matrix(map(lambda y: np.array(map(lambda x: 1 if x in y['ingredients'] else 0, cats), dtype='int8'), test))

In [13]:
import lasagne
from lasagne.layers import InputLayer, DenseLayer, DropoutLayer
from lasagne.utils import floatX
import theano.tensor as T
import theano

Using gpu device 0: GeForce GTX 670 (CNMeM is disabled)


In [42]:
input_var = T.tensor3('inputs')
target_var = T.tensor3('targets')

In [104]:
net = {}
net['input'] = InputLayer((None, 1, 6714),input_var=input_var)
net['fc1'] = DenseLayer(net['input'], num_units=200,
                        nonlinearity = lasagne.nonlinearities.sigmoid)
net['fc2'] = DenseLayer(net['fc1'], num_units=6714, nonlinearity=lasagne.nonlinearities.sigmoid, W = net['fc1'].W.T)
output_layer = net['fc2']

In [105]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt].reshape((-1, 1, 6714)), targets[excerpt]

In [106]:
prediction = lasagne.layers.get_output(output_layer).reshape((-1, 1, 6714))
loss = lasagne.objectives.squared_error(prediction, input_var)
loss = loss.mean() + lasagne.regularization.regularize_layer_params_weighted({net['fc1']: 0.1}, 
                                                                             lasagne.regularization.l2)

params = lasagne.layers.get_all_params(net['fc1'], trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.5, momentum=0.9)

test_prediction = lasagne.layers.get_output(output_layer, deterministic=True).reshape((-1, 1, 6714))
test_loss = lasagne.objectives.squared_error(test_prediction, input_var)
test_loss = test_loss.mean()

train_fn = theano.function([input_var], loss, updates=updates)

val_fn = theano.function([input_var], test_loss)

test_fn = theano.function([input_var], test_prediction)

In [37]:
from sklearn.cross_validation import train_test_split
import time

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .15)

In [39]:
X_train = X_train.toarray()
X_test = X_test.toarray()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

In [40]:
X_train.shape

(33807, 6714)

In [107]:
num_epochs = 1000
batchsize = 1000

terr = []
tacc = []
verr = []
vacc = []
eps = []

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, 1, shuffle=False):
        inputs, targets = batch
        err = val_fn(inputs)
        val_err += err
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    
    eps.append(epoch+1)
    terr.append(train_err / float(train_batches))
    verr.append(val_err / float(val_batches))

Starting training...
Epoch 1 of 1000 took 5.554s
  training loss:		3.734677
  validation loss:		0.247682
Epoch 2 of 1000 took 5.459s
  training loss:		0.250913
  validation loss:		0.247701
Epoch 3 of 1000 took 5.555s
  training loss:		0.248830
  validation loss:		0.247700
Epoch 4 of 1000 took 5.435s
  training loss:		0.248833
  validation loss:		0.247698
Epoch 5 of 1000 took 5.430s
  training loss:		0.248832
  validation loss:		0.247696
Epoch 6 of 1000 took 5.465s
  training loss:		0.248797
  validation loss:		0.247694
Epoch 7 of 1000 took 5.444s
  training loss:		0.248779
  validation loss:		0.247692
Epoch 8 of 1000 took 5.434s
  training loss:		0.248774
  validation loss:		0.247689
Epoch 9 of 1000 took 5.421s
  training loss:		0.248772
  validation loss:		0.247687
Epoch 10 of 1000 took 5.443s
  training loss:		0.248771
  validation loss:		0.247685
Epoch 11 of 1000 took 5.420s
  training loss:		0.248771
  validation loss:		0.247683
Epoch 12 of 1000 took 5.425s
  training loss:		0.2487

KeyboardInterrupt: 